In [6]:
%matplotlib inline
%load_ext autoreload
%autoreload 5
%autosave 15

import operator
import functools
import numpy as np
import scipy as sp
import pandas as pd
import math
import random
import pylab as pl
import matplotlib.pyplot as plt
import sklearn
import os
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from collections import defaultdict
from math import log
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import mean_squared_error


Autosaving every 15 seconds


In [2]:
path = "D:/Users/Daria/Code2/MachineLearning/hw3_bayes/Bayes/pu1/part"
SPAM = "spam"
LEGIT = "legit"
def getFiles(folderNumber):
    curPath = path + str(folderNumber)
    files = []
    for filename in os.listdir(curPath):
        file = open(curPath + "/" + filename, 'r')
        subject = list(map(int, file.readline().split()[1:]))
        file.readline()
        text = list(map(int, file.readline().split()))
        if LEGIT in filename:
            files.append((LEGIT, [subject, text]))
        else:
            files.append((SPAM, [subject, text]))
    return files


In [89]:
def training(trains):
    frequencies = defaultdict(lambda:0)
    classes = defaultdict(lambda:0)
    counts = defaultdict(lambda:0)
    for clazz, file in trains:
        for word in file[1]:
            # count files, not words???
            frequencies[clazz, word] += 1.0
            counts[word] += 1.0
        classes[clazz] += len(file[1])
    for clazz, word in frequencies:
        frequencies[clazz, word] /= counts[word]
        # можно размытие по лапласу, как будто каждо слово видели на 1 раз больше
#         frequencies[clazz, word] = (frequencies[clazz, word] + 1.0) / (1.0 + counts[word])

    for clazz in classes:
        classes[clazz] /= len(trains)
    return classes, frequencies

In [100]:
def trainingSubj(trains):
    frequencies = defaultdict(lambda:0)
    classes = defaultdict(lambda:0)
    counts = defaultdict(lambda:0)
    for clazz, file in trains:
        for word in file[0]:
            frequencies[clazz, word] += 1.0
            counts[word] += 1.0
        for word in file[1]:
            frequencies[clazz, word] += 1.0
            counts[word] += 1.0
        classes[clazz] += len(file[1]) + len(file[0])
    for clazz, word in frequencies:
        frequencies[clazz, word] /= counts[word]
    for clazz in classes:
        classes[clazz] /= len(trains)
    return classes, frequencies

In [119]:
errorWeight = defaultdict(lambda:1)
errorWeight[SPAM] = 1
errorWeight[LEGIT] = 1
def classify(classifier, file):
    classes, frequencies = classifier
    #P(C|o_1..o_n) = P(c)*prod(P(o_i|c)
    # result class = argmax(c, P(c)*prod(P(o_i|c))) = argmin(c, -log(P(c) - sum(log(P(o_i|c)))))
    spamProb = -log(classes[SPAM])
    legitProb = -log(classes[LEGIT])
    for word in file[1]:
        spamProb -= log(frequencies[SPAM, word] + 10 ** (-7))
        legitProb -= log(frequencies[LEGIT, word] + 10 ** (-7))
    spamProb -= log(errorWeight[SPAM])
    legitProb -= log(errorWeight[LEGIT])
    if (abs(spamProb - legitProb) < 20):
        return LEGIT
    if (spamProb < legitProb):
        return SPAM
    return LEGIT
#     return min(classes.keys(), key = lambda clazz: -log(classes[clazz]) -
#                sum(log(frequencies[clazz, word] + 10 ** (-7)) for word in file[1]))

In [120]:
folds = [getFiles(i) for i in range(1, 11)]
meanClassifier = defaultdict(lambda:0), defaultdict(lambda:0)
for trains in folds:
    curClassifier = training(trains)
    meanClassifier = getMeanClassifier(curClassifier, meanClassifier)
predict = []
answer = []
counts = [[0, 0], [0, 0]]
cnt = 0
for test in folds:
    for clazz, file in test:
        predClass = classify(meanClassifier, file)
        predict.append(int(predClass == SPAM))
        answer.append(int(clazz == SPAM))
        counts[predClass == SPAM][clazz == SPAM] += 1
        cnt += 1
print("mean squared error = ", mean_squared_error(predict, answer))
print("f1 score = ", f1_score(predict, answer))
print("test count = ", cnt)
print("SPAM is SPAM = ", counts[0][0])
print("SPAM is LEGIT = ", counts[0][1])
print("LEGIT is SPAM = ", counts[1][0])
print("LEGIT is LEGIT = ", counts[1][1])

mean squared error =  0.00366972477064
f1 score =  0.995824634656
test count =  1090
SPAM is SPAM =  609
SPAM is LEGIT =  3
LEGIT is SPAM =  1
LEGIT is LEGIT =  477


In [20]:
def getMeanClassifier(classifier1, classifier2):
    curClasses, curFrequencies = classifier1
    meanClasses, meanFrequencies = classifier2
    for clazz in curClasses:
        meanClasses[clazz] = (meanClasses[clazz] + curClasses[clazz]) / 2
    for clazz, word in curFrequencies:
        meanFrequencies[clazz, word] = (meanFrequencies[clazz, word] + curFrequencies[clazz, word]) / 2
    return meanClasses, meanFrequencies

In [ ]:
training on subject
mean squared error =  0.361467889908
f1 score =  0.679674796748
bad =  332  from  1090 ;  0.30458715596330277 %
====
training on text
mean squared error =  0.00275229357798
f1 score =  0.996884735202
test count =  1090
SPAM is SPAM =  607
SPAM is LEGIT =  0
LEGIT is SPAM =  3
LEGIT is LEGIT =  480
====
training on text with laplas
mean squared error =  0.00275229357798
f1 score =  0.996884735202
test count =  1090
SPAM is SPAM =  607
SPAM is LEGIT =  0
LEGIT is SPAM =  3
LEGIT is LEGIT =  480
====
< 20
mean squared error =  0.00366972477064
f1 score =  0.995824634656
test count =  1090
SPAM is SPAM =  609
SPAM is LEGIT =  3
LEGIT is SPAM =  1
LEGIT is LEGIT =  477
